In [1]:
import pandas as pd
import numpy as np

%store -r NSLFlows
%store -r BNDFlows
%store -r NEMFlows
%store -r ELCFlows
%store -r IF1Flows
%store -r IF2Flows
%store -r MEWFlows

In [2]:
ImportData = pd.DataFrame({"North Sea Link": NSLFlows.iloc[:,1]})
ImportData["BritNed"] = BNDFlows.iloc[:,1]
ImportData["Nemo Link"] = NEMFlows.iloc[:,1]
ImportData["ElecLink"] = ELCFlows.iloc[:,1]
ImportData["IFA"] = IF1Flows.iloc[:,1]
ImportData["IFA 2"] = IF2Flows.iloc[:,1]
ImportData["Moyle and East-West"] = MEWFlows.iloc[:,1]

ImportData["Date"] = pd.date_range(start="1/1/2021", periods=730, freq="D")

NewOrder = ["Date", "North Sea Link", "BritNed", "Nemo Link", "ElecLink", "IFA", "IFA 2", "Moyle and East-West"]

ImportData = ImportData.reindex(columns=NewOrder)

In [3]:
ImportData = ImportData.replace(np.NaN, 0)

ImportData["Total Imports"] = (ImportData["North Sea Link"] + ImportData["BritNed"]
                                     + ImportData["Nemo Link"] + ImportData["ElecLink"]
                                     + ImportData["IFA"] + ImportData["IFA 2"]
                                     + ImportData["Moyle and East-West"])
print(ImportData.to_string())

          Date  North Sea Link   BritNed  Nemo Link  ElecLink           IFA         IFA 2  Moyle and East-West  Total Imports
0   2021-01-01             0.0      6.00   24029.25       0.0      0.000000      0.000000          9007.000000   33042.250000
1   2021-01-02             0.0      0.00   23605.25       0.0      0.000000      0.000000           765.000000   24370.250000
2   2021-01-03             0.0      0.00   23762.25       0.0      0.000000      0.000000          1095.000000   24857.250000
3   2021-01-04             0.0     17.00   21801.50       0.0      0.000000      0.000000           726.000000   22544.500000
4   2021-01-05             0.0     11.00   14150.00       0.0      0.000000      0.000000           140.000000   14301.000000
5   2021-01-06             0.0     14.00   19651.50       0.0      0.000000      0.000000           422.000000   20087.500000
6   2021-01-07             0.0     15.00   22638.00       0.0      0.000000      0.000000          2739.000000   25392

In [4]:
print("Max Import [MWh]:", max(ImportData["Total Imports"]))

a = sum(ImportData["Total Imports"])
a

Max Import [MWh]: 134366.75


42582152.75

In [5]:
ELC1 = pd.DataFrame({"Index": range(1,366)})
ELC1["GB - FR"] = np.NaN

ELC2 = pd.DataFrame({"Index": range(365, 731)})
ELC2["GB - FR"] = ELCFlows.iloc[:,2]

print(ELC1)
print(ELC2)

ExportData = pd.DataFrame({"North Sea Link": NSLFlows.iloc[:,2]})
ExportData["BritNed"] = BNDFlows.iloc[:,2]
ExportData["Nemo Link"] = NEMFlows.iloc[:,2]
ExportData["ElecLink"] = pd.concat([ELC1["GB - FR"], ELC2["GB - FR"]], ignore_index=True)
ExportData["IFA"] = IF1Flows.iloc[:,2]
ExportData["IFA 2"] = IF2Flows.iloc[:,2]
ExportData["Moyle and East-West"] = MEWFlows.iloc[:,2]

ExportData["Date"] = pd.date_range(start="1/1/2021", periods=730, freq="D")

NewOrder = ["Date", "North Sea Link", "BritNed", "Nemo Link", "ElecLink", "IFA", "IFA 2", "Moyle and East-West"]

ExportData = ExportData.reindex(columns=NewOrder)
print(ExportData.to_string())

NameError: name 'ELC1' is not defined

In [ ]:
ExportData = ExportData.replace(np.NaN, 0)

ExportData["Total Exports"] = (ExportData["North Sea Link"] + ExportData["BritNed"]
                                     + ExportData["Nemo Link"] + ExportData["ElecLink"]
                                     + ExportData["IFA"] + ExportData["IFA 2"]
                                     + ExportData["Moyle and East-West"])

%store ExportData

print(ExportData.to_string())

In [ ]:
b = sum(ExportData["Total Exports"])
b

In [ ]:
from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

fig, ax = plt.subplots()
fig.set_figheight(8)
fig.set_figwidth(10)

plt.title('Total Interconnector Flows')
plt.ylabel('Daily Interconnector Flows (MWh)')

Mean = (ImportData["Total Imports"] - ExportData["Total Exports"])/math.sqrt(2)
Sum = ImportData["Total Imports"] - ExportData["Total Exports"]

ax.plot(ImportData["Date"], ImportData["Total Imports"], color="green", label="Imports")
ax.plot(ExportData["Date"], -ExportData["Total Exports"], color="red", label="Exports")
ax.plot(ImportData["Date"], Sum.rolling(28,  center=True).mean(), linewidth = 3, color = "gold", label="Rolling Average")
#ax.plot(ImportData["Date"], Mean, label="RMS", color='black')


plt.legend()
plt.grid(visible=True, which='major', color='0.65', linestyle='-')
plt.grid(visible=True, which='minor', color='0.65', linestyle=':')

ax.yaxis.get_ticklocs(minor=True)
ax.minorticks_on()

fig.autofmt_xdate()

print(ax.get_xlim())

plt.tick_params(axis='x', which='major', labelsize=10)
plt.xticks(rotation=45, ha='right')
plt.xlim(18628, 19358)
plt.tight_layout()

plt.fill_between(ImportData["Date"], ImportData["Total Imports"], color="green")
plt.fill_between(ExportData["Date"], -ExportData["Total Exports"], color="red")

ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.xaxis.set_minor_formatter(mdates.DateFormatter(""))

plt.show()